# LLM und RAG Tutorial

Die in diesem Tutorial vorgestellten Implementierungen stellen exemplarische Lösungsansätze dar. Sie dienen primär der Illustration grundlegender Konzepte und Methoden. In der praktischen Anwendung existieren diverse alternative Herangehensweisen, die je nach spezifischem Anwendungsfall, Ressourcenverfügbarkeit und Anforderungsprofil vorzuziehen sein können. Die präsentierten Beispiele sind daher als Ausgangspunkt für eigene Implementierungen zu verstehen, nicht als definitive Best Practices.

## 1. Datenerfassung

Im ersten Schritt der Implementierung des Uni-Chatbots steht die systematische Erfassung der Datenbasis. Diese bildet das Fundament für die spätere Informationsverarbeitung und determiniert maßgeblich die Qualität der generierten Antworten.

#### Primäre Datenquellen und Erfassungsmethoden:
- Webseiten
    - Crawlen
    - Inhalte herauskopierne und in separater Datei speichern
- PDFs
- Excel
- CSVs
- Datenbanken (mit ensprechenden Schnittstellen)

## 2. Konfiguration eines LLMs

Nach der Datenerfassung erfolgt die Implementierung des Large Language Models. Wir verwenden hierfür Llama in Verbindung mit der Ollama-Schnittstelle, die sowohl das Hauptsprachmodell als auch die Einbettungsfunktionalitäten bereitstellt.

In [ ]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Initialize embedding model for text vectorization
ollama_embed_model  = OllamaEmbedding(
    model_name="llama3",
)

# Set as default embedding model
Settings.embed_model = ollama_embed_model

# Configure main LLM with specific parameters
ollama_llm = Ollama(
    model="llama3:8b",
    temperature=0.1,
    context_window=8192,
    base_url='http://localhost:11434',
    request_timeout=120.0
)

# Set as default LLM
Settings.llm = ollama_llm

# Verify configuration
print(f"Embedding model: {Settings.embed_model}")
print(f"LLM: {Settings.llm}")

Embedding model: model_name='llama3' embed_batch_size=10 callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f583de56960> num_workers=None base_url='http://localhost:11434' ollama_additional_kwargs={}
LLM: callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f583de56960> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x7f583f5d9f80> completion_to_prompt=<function default_completion_to_prompt at 0x7f583f4edbc0> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None base_url='http://localhost:11434' model='llama3' temperature=0.1 context_window=8192 request_timeout=120.0 prompt_key='prompt' json_mode=False additional_kwargs={} is_function_calling_model=True keep_alive=None


### Methodische Erläuterung:
Die Konfiguration umfasst zwei zentrale Komponenten:

1. Das Embedding-Modell zur Vektorisierung textueller Daten
2. Das Hauptsprachmodell zur Generierung der Antworten

Die Parameter wurden dabei wie folgt optimiert:

- Niedrige Temperatur (0.1) für maximale Präzision
- Großes Kontextfenster für umfassende Informationsverarbeitung

### Beispielhafte Anwendung des geladenen LLMs

In [4]:
def process_questions(questions):
    """
    Process questions using the configured Ollama model directly.
    """
    for i, question in enumerate(questions, 1):
        print(f"\nFrage {i}: {question}")
        print("-" * 80)
        
        try:
            response = Settings.llm.complete(question)
            print(f"Antwort: {response}\n")
            
        except Exception as e:
            print(f"Fehler: {str(e)}\n")

# Example usage
questions = [
    "In welchen Bibliotheken kann ich Bücher ausleihen?",
    "Wie lange ist die Ausleihfrist für Bücher?",
    "Wie kann ich die Ausleihfrist meines ausgeliehenen Buches verlängern?",
    "Gibt es unterschiede für Fristen und Verlängerungen, je nachdem, wo ich das Buch ausgeliehen habe?",
    "Was passiert, wenn ich meine Ausleihfrist nicht einhalte?",
    "Wo kann ich überhaupt nach Literatur suchen?",
    "Gibt es eine Möglichkeit, Kinder-und Jugendbücher auszuleihen und wenn ja, wo?",
]

# Run the processor
process_questions(questions)


Frage 1: In welchen Bibliotheken kann ich Bücher ausleihen?
--------------------------------------------------------------------------------
Antwort: Eine gute Frage!

Es gibt viele Bibliotheken, in denen du Bücher ausleihen kannst. Hier sind einige Möglichkeiten:

1. **Öffentliche Bibliotheken**: In Deutschland gibt es eine Vielzahl öffentlicher Bibliotheken, die von den Kommunen oder Landkreisen getragen werden. Du kannst in deiner Nähe nachsehen, ob es eine öffentliche Bibliothek gibt und welche Bücher sie anbieten.
2. **Staatsbibliotheken**: Staatsbibliotheken sind spezielle Bibliotheken, die von den Regierungen oder Ministerien getragen werden. Sie bieten oft eine umfangreiche Sammlung an Büchern und anderen Medien an.
3. **Universitätsbibliotheken**: Universitätsbibliotheken sind für Studierende und Forscher gedacht, aber auch für die Öffentlichkeit zugänglich. Sie bieten oft eine breite Palette an Fachbüchern und wissenschaftlichen Publikationen an.
4. **Schulbibliotheken**: Sc

## 3. Datenaufbereitung und Transformation

Die nächste Phase beinhaltet die systematische Aufbereitung der erfassten Daten für die Verarbeitung durch das LLM.

In [5]:
from pathlib import Path

from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

# Directory structure initialization
curr_dir = Path.cwd()
root_dir = curr_dir.parent
data_library = root_dir / 'assets' / 'BIS'

print(f"Pfad zum Laden externer Daten: {data_library}")

# Document loading
documents = SimpleDirectoryReader(data_library).load_data()

# Vector index creation
index = VectorStoreIndex.from_documents(documents, show_progress=False)
query_engine = index.as_query_engine()

Pfad zum Laden externer Daten: /home/jan-hendrik/home/hobby/ReEngGP/assets/BIS


Dieser Prozess erfüllt mehrere kritische Funktionen:

- Transformation der Rohdaten in vektorielle Repräsentationen
- Aufbau einer effizienten Suchstruktur
- Optimierung der Antwortgenerierung

Jetzt kann beobachtet werden, wie sich die Beantwortung der eben gezeigten Fragen bereits verändert:

In [8]:
def process_questions(questions, query_engine):
    """
    Process questions using the configured query engine with vector store index.
    
    Args:
        questions (list): List of questions to process
        query_engine: Configured query engine instance
    """
    for i, question in enumerate(questions, 1):
        print(f"\nFrage {i}: {question}")
        print("-" * 80)
        
        try:
            response = query_engine.query(question)
            print(f"Antwort: {response}\n")
            
        except Exception as e:
            print(f"Fehler: {str(e)}\n")

process_questions(questions, query_engine)


Frage 1: Was sind Kreditpunkte und wofür benötige ich sie?
--------------------------------------------------------------------------------
Antwort: According to the provided information, there is no mention of "Kreditpunkte" or any related concept. The text only discusses library loan periods and how to extend them. Therefore, I can provide an answer based solely on this context.

It appears that Kreditpunkte are not mentioned in this context. However, if you need to extend the loan period for a book, you have three options: use your library account in ORBISplus, send a written request via email to bis-info@uni-oldenburg.de, or take no action as the initial loan period will expire.


Frage 2: Welchen Hochschulgrad habe ich nach Abschluss des Masters?
--------------------------------------------------------------------------------
Antwort: The query is asking about your academic degree after completing a Master's program. However, there is no relevant information provided in the given

## 4. Implementierung des Systemverhaltens

In dieser Phase definieren wir das normative Verhalten des Assistenzsystems durch ein detailliertes Prompttemplate. Das Verändern der Prompt kann teils drastische Auswrikungen darauf haben, wie das LLM reagiert.


In [9]:
from llama_index.core import PromptTemplate

custom_response_prompt = PromptTemplate(
    """
    WICHTIG: Antworte ausschließlich auf Deutsch

    Du bist ein KI-Assistent für die deutsche Carl von Ossietzky Universität Oldenburg. Deine Aufgabe ist es, Studierenden, Mitarbeitenden und Interessierten präzise und hilfreiche Informationen über die Universität zu geben. Beachte dabei folgende Richtlinien:

    1. Antworte immer in deutscher Sprache.
    2. Sei höflich, geduldig und respektvoll. Verwende eine formelle Anrede ("Sie"), es sei denn, der Nutzer bittet ausdrücklich um eine informelle Ansprache.
    3. Deine Antworten sollten klar, prägnant und informativ sein. Vermeide unnötig komplizierte Fachbegriffe, erkläre sie aber, wenn sie relevant sind.
    4. Wenn du eine Frage nicht beantworten kannst oder dir Informationen fehlen, sage das offen. Verweise in solchen Fällen auf offizielle Universitätsquellen wie die Website oder das Studierendensekretariat.
    5. Bei Fragen zu Studiengängen, Bewerbungsverfahren, Fristen oder Universitätseinrichtungen gib allgemeine Informationen und verweise für spezifische Details auf die zuständigen Stellen.
    6. Respektiere die Privatsphäre. Frage nicht nach persönlichen Informationen und gib keine heraus.
    7. Bei sensiblen Themen wie psychischer Gesundheit oder finanziellen Problemen verweise auf professionelle Unterstützungsangebote der Universität.
    8. Sei dir deiner Rolle als KI-Assistent bewusst. Gib keine persönlichen Meinungen oder Empfehlungen zu kontroversen Themen.
    9. Fördere eine positive Einstellung zum Studium und zur Universität, ohne dabei kritische Fragen zu ignorieren.
    10. Wenn du Informationen aus dem universitätsspezifischen Kontext gibst, weise darauf hin, dass diese möglicherweise nicht vollständig oder aktuell sind, und empfiehl, sie auf offiziellen Kanälen zu überprüfen.
    11. Wenn relevant, füge bitte Links zu offiziellen Universitätsseiten in deine Antwort ein.

    Dein Ziel ist es, eine zuverlässige und freundliche Anlaufstelle für alle universitätsbezogenen Fragen zu sein und den Nutzern dabei zu helfen, die benötigten Informationen oder den richtigen Ansprechpartner zu finden.

    WICHTIG: Antworte ausschließlich auf Deutsch. Falls deine Antwort nicht auf Deutsch ist, übersetze sie bitte sofort ins Deutsche.

    Menschliche Anfrage: {query_str}

    Relevanter Kontext:
    {context_str}

    Basierend auf dem gegebenen Kontext und unter Berücksichtigung der oben genannten Richtlinien, hier ist meine Antwort auf Deutsch:

    """
)

# Create the index
index_v2 = VectorStoreIndex.from_documents(documents, show_progress=False)
query_engine_v2 = index_v2.as_query_engine()

query_engine_v2.update_prompts(
    {"response_synthesizer:text_qa_template": custom_response_prompt}
)

In [7]:
process_questions(questions, query_engine_v2)


Frage 1: In welchen Bibliotheken kann ich Bücher ausleihen?
--------------------------------------------------------------------------------
Antwort: Hallo! Ich freue mich, Ihnen bei Ihrer Frage zu helfen. Die Carl von Ossietzky Universität Oldenburg hat insgesamt drei Bibliotheken: die Zentralbibliothek Campus Haarentor und zwei weitere Standorte in der Innenstadt.

Um Bücher ausleihen zu können, müssen Sie sich zunächst registrieren oder ein bestehendes Bibliothekskonto haben. Danach können Sie in den Bibliotheken Medien entweder persönlich abholen oder per ORBISplus online ausleihen.

Wenn Sie bereits ein Buch ausgeliehen haben und die Leihfrist verlängern möchten, können Sie dies über Ihr Bibliothekskonto in ORBISplus oder schriftlich per E-Mail an bis-info@uni-oldenburg.de tun. Telefonische Fristverlängerungen sind leider nicht möglich.

Ich hoffe, diese Informationen helfen Ihnen bei Ihrer Suche nach Büchern in den Bibliotheken der Carl von Ossietzky Universität Oldenburg. Wenn 

## 5. Integration der Prüfungsordnungen

Die Integration der Prüfungsordnungen erfolgt zunächst modular für den Master StudienWirtschaftsinformatikgänge, dann mit weiteren Studiengängen.

In [10]:
# path to data
data_po = root_dir / 'assets' / 'PO' / 'master' / 'wirtschaftsinformatik'

print(f"Pfad zum Laden externer Daten: {data_po}")

documents = SimpleDirectoryReader(data_po).load_data()
index = VectorStoreIndex.from_documents(documents, show_progress=False)
query_engine_v3 = index.as_query_engine()
query_engine_v3.update_prompts(
    {"response_synthesizer:text_qa_template": custom_response_prompt}
)

Pfad zum Laden externer Daten: /home/jan-hendrik/home/hobby/ReEngGP/assets/PO/master/wirtschaftsinformatik
Failed to load file /home/jan-hendrik/home/hobby/ReEngGP/assets/PO/master/wirtschaftsinformatik/Anzeige_einer_Prüfungsunfähigkeit_Formular.pdf with error: RetryError[<Future at 0x7fd561adb050 state=finished raised DependencyError>]. Skipping...
Failed to load file /home/jan-hendrik/home/hobby/ReEngGP/assets/PO/master/wirtschaftsinformatik/Formular_Anmeldung_Abschlussarbeit.pdf with error: RetryError[<Future at 0x7fd561ec51c0 state=finished raised DependencyError>]. Skipping...


In [10]:
questions_v2 = [
    "Was sind Kreditpunkte und wofür benötige ich sie?",
    "Welchen Hochschulgrad habe ich nach Abschluss des Masters?",
    "Was ist Dauer, Umfang des Masterstudiums?",
    "Wie ist eine Prüfung aufgebaut und welche Arten der Prüfungsleistung?",
]

process_questions(questions_v2, query_engine_v3)


Frage 1: Was sind Kreditpunkte und wofür benötige ich sie?
--------------------------------------------------------------------------------
Antwort: Liebe/r Studierende,

Kreditpunkte sind eine Art von Punkten, die du in deinem Masterstudium sammelst. Sie werden anhand bestimmter Kriterien wie Prüfungsleistungen, Teilnahmen an Seminaren und anderen Aktivitäten vergeben. Die Kreditpunkte dienen dazu, deine Leistung im Studium zu messen und zu bewerten.

Um den Masterabschluss zu erreichen, musst du insgesamt 120 Kreditpunkte sammeln. Es ist also wichtig, dass du regelmäßig Prüfungen bestehst und an Seminaren teilnimmst, um deine Kreditpunkte aufzubauen.

Es ist auch zu beachten, dass die Bestnote ohne Bonusleistungen erreicht werden kann. Das bedeutet, dass du nicht notwendigerweise alle Prüfungen mit der höchsten Note bestehen musst, um den Masterabschluss zu erreichen.

Ich hoffe, diese Informationen helfen dir bei deinem Studium. Wenn du weitere Fragen hast, stehe ich gerne zur Verf

### Inklusive anderer Prüfungsordnungen

In [11]:
data_po_v2 = root_dir / 'assets' / 'PO' / 'master'

print(f"Pfad zum Laden externer Daten: {data_po_v2}")

documents = SimpleDirectoryReader(data_po, recursive=True).load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine_v4 = index.as_query_engine()
query_engine_v4.update_prompts(
    {"response_synthesizer:text_qa_template": custom_response_prompt}
)

Pfad zum Laden externer Daten: /home/jan-hendrik/home/hobby/ReEngGP/assets/PO/master
Failed to load file /home/jan-hendrik/home/hobby/ReEngGP/assets/PO/master/wirtschaftsinformatik/Anzeige_einer_Prüfungsunfähigkeit_Formular.pdf with error: RetryError[<Future at 0x7fe817affa40 state=finished raised DependencyError>]. Skipping...
Failed to load file /home/jan-hendrik/home/hobby/ReEngGP/assets/PO/master/wirtschaftsinformatik/Formular_Anmeldung_Abschlussarbeit.pdf with error: RetryError[<Future at 0x7fe826d489e0 state=finished raised DependencyError>]. Skipping...


In [12]:
process_questions(questions_v2, query_engine_v4)


Frage 1: Was sind Kreditpunkte und wofür benötige ich sie?
--------------------------------------------------------------------------------
Antwort: Liebe/r Studierende,

Kreditpunkte sind eine Art von Punkten, die in Deutschland für das Erreichen bestimmter Leistungen in einem Studiengang vergeben werden. Im Falle des Masterstudiengangs Wirtschaftsinformatik an der Carl von Ossietzky Universität Oldenburg werden Kreditpunkte (CP) für die erfolgreiche Absolvierung von Modulen und Prüfungen vergeben.

Die Kreditpunkte dienen dazu, den Studierenden ein Maß für ihre Leistungen zu geben und ihnen eine Vorstellung davon zu vermitteln, wie gut sie in einem bestimmten Studiengang abschneiden. Die Kreditpunkte werden auf der Grundlage von Noten und Prüfungsleistungen vergeben.

Es ist wichtig zu beachten, dass die Bestnote auch ohne Bonusleistungen erreicht werden kann. Dies bedeutet, dass ein Studierender trotzdem eine gute Note erzielen kann, wenn er sich nicht an bestimmten Aktivitäten ode

## 6. Individualisierung der Systemantworten

Abschließend implementieren wir die Personalisierungsfunktionalität des Systems.


In [11]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, PromptTemplate
from typing import Dict

def create_personalized_prompt(student_info: Dict[str, str]) -> PromptTemplate:
    return PromptTemplate(
    f"""
    WICHTIG: Antworte ausschließlich auf Deutsch

    Du bist ein KI-Assistent für die deutsche Carl von Ossietzky Universität Oldenburg. Du sprichst gerade mit einem Studenten mit folgenden Informationen:
    
    Name: {student_info.get('name', 'Nicht angegeben')}
    Studiengang: {student_info.get('studiengang', 'Nicht angegeben')}
    Grad: {student_info.get('grad', 'Nicht angegeben')}
    Semester: {student_info.get('semester', 'Nicht angegeben')}
    Matrikelnummer: {student_info.get('matrikelnummer', 'Nicht angegeben')}
    Fakultät: {student_info.get('fakultaet', 'Nicht angegeben')}

    Benutze diese Informationen, um deine Antworten zu personalisieren und relevanter zu gestalten. Beachte dabei folgende Richtlinien:

    1. Antworte immer in deutscher Sprache.
    2. Sei höflich, geduldig und respektvoll. Verwende eine formelle Anrede ("Sie"), es sei denn, der Nutzer bittet ausdrücklich um eine informelle Ansprache.
    3. Deine Antworten sollten klar, prägnant und informativ sein. Vermeide unnötig komplizierte Fachbegriffe, erkläre sie aber, wenn sie relevant sind.
    4. Wenn du eine Frage nicht beantworten kannst oder dir Informationen fehlen, sage das offen. Verweise in solchen Fällen auf offizielle Universitätsquellen wie die Website oder das Studierendensekretariat.
    5. Bei Fragen zu Studiengängen, Bewerbungsverfahren, Fristen oder Universitätseinrichtungen gib allgemeine Informationen und verweise für spezifische Details auf die zuständigen Stellen.
    6. Respektiere die Privatsphäre. Frage nicht nach weiteren persönlichen Informationen und gib keine heraus.
    7. Bei sensiblen Themen wie psychischer Gesundheit oder finanziellen Problemen verweise auf professionelle Unterstützungsangebote der Universität.
    8. Sei dir deiner Rolle als KI-Assistent bewusst. Gib keine persönlichen Meinungen oder Empfehlungen zu kontroversen Themen.
    9. Fördere eine positive Einstellung zum Studium und zur Universität, ohne dabei kritische Fragen zu ignorieren.
    10. Wenn du Informationen aus dem universitätsspezifischen Kontext gibst, weise darauf hin, dass diese möglicherweise nicht vollständig oder aktuell sind, und empfiehl, sie auf offiziellen Kanälen zu überprüfen.
    11. Wenn relevant, füge bitte Links zu offiziellen Universitätsseiten in deine Antwort ein.
    12. Beziehe dich in deinen Antworten auf die spezifischen Informationen des Studenten, wenn es angemessen ist.

    Dein Ziel ist es, eine zuverlässige und freundliche Anlaufstelle für alle universitätsbezogenen Fragen zu sein und den Nutzern dabei zu helfen, die benötigten Informationen oder den richtigen Ansprechpartner zu finden.

    Menschliche Anfrage: {{query_str}}

    Relevanter Kontext:
    {{context_str}}

    Basierend auf dem gegebenen Kontext, den Studenteninformationen und unter Berücksichtigung der oben genannten Richtlinien, hier ist meine personalisierte Antwort auf Deutsch:
    """
    )

def create_personalized_query_engine(index: VectorStoreIndex, student_info: Dict[str, str]):
    query_engine = index.as_query_engine()
    personalized_prompt = create_personalized_prompt(student_info)
    query_engine.update_prompts(
        {"response_synthesizer:text_qa_template": personalized_prompt}
    )
    return query_engine

- Kontextuelle Anpassung der Informationsbereitstellung
- Optimierung der Antwortqualität durch Berücksichtigung individueller Parameter

In [12]:
# random student info
student_info = {
    "name": "Max Mustermann",
    "studiengang": "Wirtschaftsinformatik",
    "grad": "Master",
    "semester": "3",
    "matrikelnummer": "123456",
    "fakultaet": "II - Informatik, Wirtschafts- und Rechtswissenschaften"
}

# load data
documents = SimpleDirectoryReader(data_po, recursive=True).load_data()
# Create the index
index = VectorStoreIndex.from_documents(documents)

personalized_query_engine = create_personalized_query_engine(index, student_info)

Failed to load file /home/jan-hendrik/home/hobby/ReEngGP/assets/PO/master/wirtschaftsinformatik/Anzeige_einer_Prüfungsunfähigkeit_Formular.pdf with error: RetryError[<Future at 0x7fd56237fe90 state=finished raised DependencyError>]. Skipping...
Failed to load file /home/jan-hendrik/home/hobby/ReEngGP/assets/PO/master/wirtschaftsinformatik/Formular_Anmeldung_Abschlussarbeit.pdf with error: RetryError[<Future at 0x7fd561ec7c50 state=finished raised DependencyError>]. Skipping...


In [13]:
questions = [
    "Was sind Kreditpunkte und wofür benötige ich sie?",
    "Welchen Hochschulgrad habe ich nach Abschluss des Masters?",
    "Was ist Dauer, Umfang des Masterstudiums?",
    "Wie ist eine Prüfung aufgebaut und welche Arten der Prüfungsleistung?",
]

for question in questions:
    response = personalized_query_engine.query(question)
    print(f"Frage: {question}")
    print(f"Antwort: {response}\n")

Frage: Was sind Kreditpunkte und wofür benötige ich sie?
Antwort: Hallo Max Mustermann,

Ich freue mich, dass du nachfragst, was Kreditpunkte sind und wofür du sie benötigst. Als Student im Master-Studiengang Wirtschaftsinformatik an der Carl von Ossietzky Universität Oldenburg bist du bestrebt, deine Studienleistungen erfolgreich abzuschließen.

Kreditpunkte (KP) sind ein Maß für die Leistung in einem Modul oder einer Prüfung. Sie werden auf Basis deiner erbrachten Leistungen und der Bewertungskriterien des jeweiligen Moduls vergeben. Die Kreditpunkte dienen dazu, deine Studienleistungen zu dokumentieren und zu bewerten.

In deinem Master-Studiengang Wirtschaftsinformatik benötigst du insgesamt 120 KP, um den Studiengang erfolgreich abzuschließen. Diese Kreditpunkte werden in verschiedenen Modulen und Prüfungen vergeben. Die Kreditpunkte sind ein wichtiger Aspekt deines Studiums, da sie deine Leistung dokumentieren und die Erfüllung der Anforderungen an den Master-Studiengang überprüf

In [14]:
response = personalized_query_engine.query("Gint es unterschiede zwischen dem Master Wirtscahftsinformatik und dem Master Germanistik?")
print(response)

Hallo Max Mustermann,

vielen Dank für deine Frage. Ich bin froh, dass du nachfragst, ob es Unterschiede zwischen dem Master Wirtschaftsinformatik und dem Master Germanistik gibt.

Zunächst möchte ich betonen, dass beide Studiengänge an der Carl von Ossietzky Universität Oldenburg angeboten werden. Der Master Wirtschaftsinformatik ist ein interdisziplinärer Studiengang, der sich auf die Verbindung von Informatik und Wirtschaft konzentriert. Er richtet sich an Studierende, die eine solide Grundlage in Informatik und Wirtschaft haben und sich für die Anwendung von IT-Lösungen in wirtschaftlichen Kontexten interessieren.

Der Master Germanistik hingegen ist ein Studiengang, der sich auf die Sprache, Literatur und Kultur des Deutschen konzentriert. Er richtet sich an Studierende, die eine solide Grundlage in Germanistik haben und sich für die Analyse und Interpretation von Texten, die in deutscher Sprache verfasst sind, interessieren.

Insgesamt gibt es keine direkte Beziehung zwischen bei

## Zusammenfassung
Die vorgestellte Implementierung demonstriert einen möglichen Ansatz zur Entwicklung eines LLM-basierten Universitätsassistenzsystems. Dabei ist zu betonen:

1. Methoden:

- Die gezeigten Implementierungen sind exemplarisch
- Alternative Ansätze können je nach Kontext vorteilhaft sein
- Kontinuierliche Evaluation und Anpassung sind essentiell


2. Entwicklungsperspektiven:

- Optimierung der Datenverarbeitung
- Erweiterung der Personalisierungsmöglichkeiten


3. Kritische Aspekte:

- Performance-Abwägungen verschiedener Implementierungen
- Skalierbarkeitsüberlegungen
- Datenschutz und Sicherheitsaspekte



Für die praktische Implementierung empfiehlt sich:

- Gründliche Evaluation der spezifischen Anforderungen
- Experimentelle Erprobung verschiedener Ansätze
- Kontinuierliche Optimierung basierend auf eigenem Feedback oder Nutzerfeedback eurer Komilitonen

Die vorgestellten Beispiele sollten als Ausgangspunkt für eigene, an den spezifischen Anwendungsfall angepasste Implementierungen verstanden werden.